In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
from datetime import datetime
import folium
import folium.plugins as plugins
from sklearn.cluster import KMeans
import seaborn as sns; sns.set()
import geopy.distance
import math
from pandas import option_context
import json
from collections import Counter

In [10]:
data = pd.read_csv("./data/pickup_delivery_occurences_every_hour.csv", index_col=0)

In [14]:
data.head()

,Zone,Period,Pickup occurences,Delivery occurences
0,1,0,"{""0"": 362, ""1"": 3}","{""0"": 362, ""1"": 3}"
1,1,1,"{""0"": 362, ""1"": 3}","{""0"": 360, ""1"": 4, ""2"": 1}"
2,1,2,"{""0"": 363, ""1"": 2}","{""0"": 361, ""1"": 3, ""2"": 1}"
3,1,3,"{""0"": 364, ""1"": 1}","{""0"": 364, ""1"": 1}"
4,1,4,"{""0"": 365}","{""0"": 364, ""1"": 1}"


In [15]:
with option_context('display.max_colwidth', 400):
    display(data.loc[(data['Zone'] == 126)])

,Zone,Period,Pickup occurences,Delivery occurences
3000,126,0,"{""0"": 329, ""1"": 26, ""2"": 9, ""3"": 1}","{""0"": 315, ""1"": 44, ""2"": 6}"
3001,126,1,"{""0"": 334, ""1"": 23, ""2"": 6, ""3"": 2}","{""0"": 333, ""1"": 29, ""2"": 2, ""3"": 1}"
3002,126,2,"{""0"": 333, ""1"": 27, ""2"": 4, ""3"": 1}","{""0"": 344, ""1"": 15, ""2"": 6}"
3003,126,3,"{""0"": 338, ""1"": 22, ""2"": 5}","{""0"": 354, ""1"": 9, ""2"": 2}"
3004,126,4,"{""0"": 348, ""1"": 13, ""2"": 4}","{""0"": 358, ""1"": 7}"
3005,126,5,"{""0"": 310, ""1"": 48, ""2"": 6, ""3"": 1}","{""0"": 355, ""1"": 10}"
3006,126,6,"{""0"": 229, ""1"": 102, ""2"": 24, ""3"": 7, ""4"": 3}","{""0"": 320, ""1"": 42, ""2"": 3}"
3007,126,7,"{""0"": 193, ""1"": 101, ""2"": 50, ""3"": 17, ""4"": 4}","{""0"": 289, ""1"": 67, ""2"": 9}"
3008,126,8,"{""0"": 224, ""1"": 92, ""2"": 41, ""3"": 5, ""4"": 3}","{""0"": 292, ""1"": 64, ""2"": 8, ""4"": 1}"
3009,126,9,"{""0"": 266, ""1"": 82, ""2"": 13, ""3"": 4}","{""0"": 276, ""1"": 79, ""2"": 9, ""3"": 1}"


In [11]:
data['Pickup distribution'] = np.nan
data['Max value deliveries'] = np.nan
data['Pickup distribution length match'] = np.nan
len(data)

6096

In [17]:
data = pd.read_csv("./data/pickup_delivery_occurences_every_hour.csv", index_col=0)

data['Pickup distribution'] = np.nan
data['Delivery distribution'] = np.nan

# pickups
for i in range(0,6096):
    
    d = json.loads(data.iloc[i,2])
    keys = list(range(int(max(d, key=int))+1))
    change = False
    for key in keys:
        if (str(key) not in d):
            change = True
            d[str(key)] = 0
    json_d = json.dumps(d)
    if change:
        data.iloc[i,2] = str(json_d)
        
        out_list = [round(d[str(k)]/365,3) for k in keys]
        data.iloc[i,4] = str(out_list)
    
    else:
        out_list = [round(d[str(k)]/365, 3) for k in keys]
        data.iloc[i,4] = str(out_list)

        
# deliveries
for i in range(0,6096):
    
    d = json.loads(data.iloc[i,3])
    keys = list(range(int(max(d, key=int))+1))
    change = False
    for key in keys:
        if (str(key) not in d):
            change = True
            d[str(key)] = 0
    json_d = json.dumps(d)
    if change:
        data.iloc[i,3] = str(json_d)
        
        out_list = [round(d[str(k)]/365,3) for k in keys]
        data.iloc[i,5] = str(out_list)
    # Fill pickup distribution
    else:
        out_list = [round(d[str(k)]/365, 3) for k in keys]
        data.iloc[i,5] = str(out_list)
    
    

In [30]:
with option_context('display.max_colwidth', 400):
    display(data.head(3070))

,Zone,Period,Pickup occurences,Delivery occurences,Pickup distribution,Delivery distribution
0,1,0,"{""0"": 362, ""1"": 3}","{""0"": 362, ""1"": 3}","[0.992, 0.008]","[0.992, 0.008]"
1,1,1,"{""0"": 362, ""1"": 3}","{""0"": 360, ""1"": 4, ""2"": 1}","[0.992, 0.008]","[0.986, 0.011, 0.003]"
2,1,2,"{""0"": 363, ""1"": 2}","{""0"": 361, ""1"": 3, ""2"": 1}","[0.995, 0.005]","[0.989, 0.008, 0.003]"
3,1,3,"{""0"": 364, ""1"": 1}","{""0"": 364, ""1"": 1}","[0.997, 0.003]","[0.997, 0.003]"
4,1,4,"{""0"": 365}","{""0"": 364, ""1"": 1}",[1.0],"[0.997, 0.003]"
...,...,...,...,...,...,...
3065,128,17,"{""0"": 160, ""1"": 134, ""2"": 39, ""3"": 21, ""4"": 10, ""6"": 1, ""5"": 0}","{""0"": 164, ""1"": 104, ""2"": 64, ""3"": 22, ""4"": 8, ""5"": 2, ""6"": 1}","[0.438, 0.367, 0.107, 0.058, 0.027, 0.0, 0.003]","[0.449, 0.285, 0.175, 0.06, 0.022, 0.005, 0.003]"
3066,128,18,"{""0"": 194, ""1"": 100, ""2"": 47, ""3"": 18, ""4"": 5, ""5"": 1}","{""0"": 176, ""1"": 109, ""2"": 53, ""3"": 19, ""4"": 7, ""5"": 1}","[0.532, 0.274, 0.129, 0.049, 0.014, 0.003]","[0.482, 0.299, 0.145, 0.052, 0.019, 0.003]"
3067,128,19,"{""0"": 218, ""1"": 107, ""2"": 30, ""3"": 7, ""4"": 3}","{""0"": 203, ""1"": 110, ""2"": 34, ""3"": 15, ""4"": 3}","[0.597, 0.293, 0.082, 0.019, 0.008]","[0.556, 0.301, 0.093, 0.041, 0.008]"
3068,128,20,"{""0"": 248, ""1"": 94, ""2"": 17, ""3"": 5, ""4"": 1}","{""0"": 223, ""1"": 114, ""2"": 19, ""3"": 9}","[0.679, 0.258, 0.047, 0.014, 0.003]","[0.611, 0.312, 0.052, 0.025]"


In [31]:
#data.to_csv("./data/pickup_delivery_distributions_every_hour.csv")

In [32]:
data = pd.read_csv("./data/pickup_delivery_distributions_every_hour.csv", index_col=0)

In [41]:
with option_context('display.max_colwidth', 400):
    display(data.head(4470))


,Zone,Period,Pickup occurences,Delivery occurences,Pickup distribution,Delivery distribution
0,1,0,"{""0"": 362, ""1"": 3}","{""0"": 362, ""1"": 3}","[0.992, 0.008]","[0.992, 0.008]"
1,1,1,"{""0"": 362, ""1"": 3}","{""0"": 360, ""1"": 4, ""2"": 1}","[0.992, 0.008]","[0.986, 0.011, 0.003]"
2,1,2,"{""0"": 363, ""1"": 2}","{""0"": 361, ""1"": 3, ""2"": 1}","[0.995, 0.005]","[0.989, 0.008, 0.003]"
3,1,3,"{""0"": 364, ""1"": 1}","{""0"": 364, ""1"": 1}","[0.997, 0.003]","[0.997, 0.003]"
4,1,4,"{""0"": 365}","{""0"": 364, ""1"": 1}",[1.0],"[0.997, 0.003]"
...,...,...,...,...,...,...
4465,187,1,"{""0"": 335, ""1"": 29, ""2"": 1}","{""0"": 326, ""1"": 35, ""2"": 3, ""3"": 1}","[0.918, 0.079, 0.003]","[0.893, 0.096, 0.008, 0.003]"
4466,187,2,"{""0"": 332, ""1"": 32, ""2"": 1}","{""0"": 345, ""1"": 18, ""2"": 2}","[0.91, 0.088, 0.003]","[0.945, 0.049, 0.005]"
4467,187,3,"{""0"": 343, ""1"": 21, ""2"": 1}","{""0"": 353, ""1"": 10, ""2"": 2}","[0.94, 0.058, 0.003]","[0.967, 0.027, 0.005]"
4468,187,4,"{""0"": 352, ""1"": 12, ""3"": 1, ""2"": 0}","{""0"": 350, ""1"": 14, ""3"": 1, ""2"": 0}","[0.964, 0.033, 0.0, 0.003]","[0.959, 0.038, 0.0, 0.003]"


In [42]:
morning_data = data.loc[(data['Period']==7) |(data['Period']==8) |(data['Period']==9)]

In [48]:
pd.options.display.max_rows = 600
with option_context('display.max_colwidth', 400):
    display(morning_data.head(600))

,Zone,Period,Pickup occurences,Delivery occurences,Pickup distribution,Delivery distribution
7,1,7,"{""0"": 352, ""1"": 11, ""2"": 2}","{""0"": 324, ""1"": 40, ""2"": 1}","[0.964, 0.03, 0.005]","[0.888, 0.11, 0.003]"
8,1,8,"{""0"": 357, ""1"": 7, ""2"": 1}","{""0"": 322, ""1"": 39, ""2"": 4}","[0.978, 0.019, 0.003]","[0.882, 0.107, 0.011]"
9,1,9,"{""0"": 354, ""1"": 11}","{""0"": 336, ""1"": 25, ""2"": 4}","[0.97, 0.03]","[0.921, 0.068, 0.011]"
31,2,7,"{""0"": 350, ""1"": 14, ""2"": 1}","{""0"": 356, ""1"": 9}","[0.959, 0.038, 0.003]","[0.975, 0.025]"
32,2,8,"{""0"": 358, ""1"": 7}","{""0"": 357, ""1"": 7, ""2"": 1}","[0.981, 0.019]","[0.978, 0.019, 0.003]"
33,2,9,"{""0"": 357, ""1"": 8}","{""0"": 360, ""1"": 4, ""2"": 1}","[0.978, 0.022]","[0.986, 0.011, 0.003]"
55,3,7,"{""0"": 343, ""1"": 20, ""2"": 1, ""3"": 1}","{""0"": 356, ""1"": 9}","[0.94, 0.055, 0.003, 0.003]","[0.975, 0.025]"
56,3,8,"{""0"": 344, ""1"": 20, ""3"": 1, ""2"": 0}","{""0"": 357, ""1"": 6, ""2"": 2}","[0.942, 0.055, 0.0, 0.003]","[0.978, 0.016, 0.005]"
57,3,9,"{""0"": 354, ""1"": 10, ""2"": 1}","{""0"": 349, ""1"": 13, ""2"": 3}","[0.97, 0.027, 0.003]","[0.956, 0.036, 0.008]"
79,4,7,"{""0"": 345, ""1"": 19, ""2"": 1}","{""0"": 346, ""1"": 19}","[0.945, 0.052, 0.003]","[0.948, 0.052]"


## Create dataset with zone coordinates

In [9]:
#zone_data = pd.read_csv("./data/2019_vy_cleaned_prepared.csv")

In [10]:
zone_data.head()

,DateTime,Trip ID,TripStage,Long,Lat,Car ID,Trip duration (min),Zone,No. trips zone
0,2018-12-31 11:39:21.888000000,1,Start,10.73382,59.92934,V5a15A3226E5F7,1403.350000,109.0,2482.0
1,2019-01-01 11:02:43.008000000,1,End,10.74900,59.91063,V5a15A3226E5F7,1403.350000,189.0,6666.0
2,2019-01-01 08:18:08.352000000,2,Start,10.73561,59.93056,V5a15A3226E89D,24.716667,109.0,2482.0
3,2019-01-01 08:42:50.976000000,2,End,10.80917,59.94132,V5a15A3226E89D,24.716667,64.0,999.0
4,2019-01-01 14:34:39.360000000,3,Start,10.72983,59.92513,V5a15A3226E999,1067.866667,129.0,3072.0


In [ ]:
a